In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
data_path = '/media/jaume/DATA/Data/Urblauna_SFTP/UKB_Cardiac_BIDS'
derivatives_path = os.path.join(data_path, 'derivatives')

# Metadata file
metadata_filename = os.path.join(derivatives_path, 'metadata_participants_ALL.tsv')
df_metadata = pd.read_csv(metadata_filename, index_col=0, sep='\t')
df_metadata

/tmp/ipykernel_1339569/2326861573.py:6: DtypeWarning: Columns (212,213,214,215,216) have mixed types. Specify dtype option on import or set low_memory=False.
  df_metadata = pd.read_csv(metadata_filename, index_col=0, sep='\t')


,Age_last_oral_contraceptive_0.0,Sex_0.0,Pacemaker_0.0,Ethnicity_0.0,BMI_0.0,Weight_0.0,Age_0.0,Townsend_deprivation_index_0.0,Age_last_oral_contraceptive_2.0,Pacemaker_2.0,...,hp,hf,diabetes,sf_af,sf_hp,sf_hf,sf_diabetes,Age_Death,Date_Death,has_img
Subject,,,,,,,,,,,,,,,,,,,,,
sub-1000101,27.0,0.0,0.0,1001.0,32.0444,72.1,63.0,-4.98,28.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
sub-1000119,NaN,1.0,0.0,1001.0,26.0604,83.5,49.0,-5.05,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
sub-1000158,NaN,1.0,0.0,1001.0,28.4429,82.2,58.0,-5.01,NaN,0.0,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,1.0
sub-1000243,NaN,1.0,0.0,1001.0,35.9631,99.1,62.0,-3.43,NaN,0.0,...,2.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,1.0
sub-1000372,NaN,1.0,0.0,1001.0,27.2693,86.4,52.0,-4.76,NaN,0.0,...,2.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sub-6023555,24.0,0.0,0.0,1003.0,23.5750,59.6,48.0,-3.71,21.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
sub-6023657,NaN,0.0,0.0,1001.0,28.1588,73.9,58.0,-0.34,NaN,0.0,...,NaN,NaN,-1.0,NaN,NaN,NaN,2.0,NaN,NaN,1.0
sub-6023670,32.0,0.0,0.0,1001.0,26.1178,67.7,49.0,-3.75,32.0,0.0,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,1.0


In [3]:

def drop_high_nan(df, threshold=0.9):
    """
    Drops columns from df that have a fraction of NaN values above the given threshold.
    
    Args:
        df (pd.DataFrame): Input DataFrame.
        threshold (float): Fraction threshold (default is 0.9, i.e. drop columns with >90% NaNs).
    
    Returns:
        pd.DataFrame: DataFrame with high-NaN columns removed.
    """
    # Compute fraction of NaN per column and drop those above threshold.
    nan_fraction = df.isna().mean()
    cols_to_keep = nan_fraction[nan_fraction < threshold].index
    return df.loc[:, cols_to_keep]

def process_columns_for_rf(df, columns_to_drop, columns_to_code, nan_threshold=0.9):
    """
    Processes a DataFrame for RandomForest modeling.
    1. Drops columns specified in columns_to_drop.
    2. Drops any columns with > nan_threshold fraction of NaN values.
    3. For each column in columns_to_code:
         - Tries to convert to numeric.
         - If conversion fails, converts the column to categorical and replaces with its codes.
    
    Args:
        df (pd.DataFrame): Input DataFrame.
        columns_to_drop (list): List of column names to drop.
        columns_to_code (list): List of column names that require processing:
                                 try numeric conversion, else convert to categorical codes.
        nan_threshold (float): Fraction threshold to drop columns with too many NaNs (default=0.9).
    
    Returns:
        pd.DataFrame: Processed DataFrame with only numeric columns.
    """
    # Drop specified columns.
    df = df.drop(columns=columns_to_drop, errors='ignore')
    
    # Drop columns with too many NaN values.
    df = drop_high_nan(df, threshold=nan_threshold)
    
    # for col in columns_to_code:
    for col in df.columns:
        if col in df.columns:
            try:
                # Try converting to numeric (float).
                df[col] = pd.to_numeric(df[col])
            except Exception:
                # If conversion fails, convert to categorical and use its codes.
                if col != 'Subject':
                    df[col] = pd.Categorical(df[col]).codes.astype(float)
        else:
            print(f"Warning: Column '{col}' not found in DataFrame.")
    
    return df

In [4]:
columns_to_drop = ['VisitDate_0', 'VisitDate_1', 'VisitDate_2', 'VisitDate_3', 'birthday',  'DBP_2.1',  'SBP_2.1',  'DBP_0.1', 'SBP_0.1',  'PulseRate_0.1', 'PulseRate_2.1',
                   'pwa_heart_rate_2.1', 'pwa_heart_rate_2.2', 'pwa_heart_rate_2.3', 'pwa_heart_rate_2.4',
                   'pwa_brachial_sbp_2.1', 'pwa_brachial_sbp_2.2', 'pwa_brachial_sbp_2.3', 'pwa_brachial_sbp_2.4',
                   'pwa_brachial_dbp_2.1', 'pwa_brachial_dbp_2.2', 'pwa_brachial_dbp_2.3', 'pwa_brachial_dbp_2.4',
                   'pwa_peripheral_pulse_pressure_2.1', 'pwa_peripheral_pulse_pressure_2.2', 'pwa_peripheral_pulse_pressure_2.3', 'pwa_peripheral_pulse_pressure_2.4',
                   'pwa_central_sbp_2.1', 'pwa_central_sbp_2.2', 'pwa_central_sbp_2.3', 'pwa_central_sbp_2.4',
                   'pwa_central_pulse_pressure_2.1', 'pwa_central_pulse_pressure_2.2', 'pwa_central_pulse_pressure_2.3', 'pwa_central_pulse_pressure_2.4',
                   'pwa_number_beats_2.1', 'pwa_number_beats_2.2', 'pwa_number_beats_2.3', 'pwa_number_beats_2.4',
                   'pwa_central_augmentation_pressure_2.1', 'pwa_central_augmentation_pressure_2.2', 'pwa_central_augmentation_pressure_2.3', 'pwa_central_augmentation_pressure_2.4',
                   'pwa_cardiac_output_2.1', 'pwa_cardiac_output_2.2', 'pwa_cardiac_output_2.3', 'pwa_cardiac_output_2.4',
                   'pwa_end_sp_2.1', 'pwa_end_sp_2.2', 'pwa_end_sp_2.3', 'pwa_end_sp_2.4',
                   'pwa_end_sp_index_2.1', 'pwa_end_sp_index_2.2', 'pwa_end_sp_index_2.3', 'pwa_end_sp_index_2.4',
                   'pwa_total_peripheral_resistance_2.1', 'pwa_total_peripheral_resistance_2.2', 'pwa_total_peripheral_resistance_2.3', 'pwa_total_peripheral_resistance_2.4',
                   'pwa_stroke_volume_2.1', 'pwa_stroke_volume_2.2', 'pwa_stroke_volume_2.3', 'pwa_stroke_volume_2.4',
                   'pwa_mean_arterial_pressure_2.1', 'pwa_mean_arterial_pressure_2.2', 'pwa_mean_arterial_pressure_2.3', 'pwa_mean_arterial_pressure_2.4',
                   'pwa_cardiac_index_2.1', 'pwa_cardiac_index_2.2', 'pwa_cardiac_index_2.3', 'pwa_cardiac_index_2.4',
                   'spiro_fvc_0.1', 'spiro_fvc_0.2', 'spiro_pef_0.1', 'spiro_pef_0.2', 
                   'spiro_fvc_2.1', 'spiro_fvc_2.2',  'spiro_qc_0.0',  'spiro_pef_2.1', 'spiro_pef_2.2', 'has_img',
                   'spiro_fvc_0.0', 'spiro_pef_0.0',
                   'Weight', 'BMI_2.0', 'Age_0.0',
                   ]

columns_to_code = ['Sex_0.0', 'Pacemaker_0.0', 'Ethnicity_0.0', 'Long_standing_illness_disability_0.0',  'Long_standing_illness_disability_2.0',
                   'snoring_0.0', 'snoring_2.0', 'had_menopause_0.0', 'ever_had_miscarriage_stillbirths_or_termination_0.0', 'ever_taken_oral_contraceptive_pill_0.0', 
                   'ever_used_hormone_replacement_therapy_0.0', 'bilateral_oophorectomy_0.0', 'ever_had_hysterectomy_0.0', 'menstruating_today_0.0',
                   'menstruating_today_2.0', 'ever_had_hysterectomy_2.0', 'bilateral_oophorectomy_2.0', 'ever_used_hormone_replacement_therapy_2.0',
                   'af', 'dcm', 'dyspnea', 'hp', 'hf', 'diabetes', 'sf_af', 'sf_hp', 'sf_hf', 'sf_diabetes'
                   'art_stiff_absence_notch_0.0', 'art_stiff_absence_notch_2.0',
                   'spiro_caffeine_last_hour_0.0', 'spiro_inhaler_last_hour_0.0', 'spiro_smoked_last_hour_0.0',
                   'spiro_caffeine_last_hour_2.0', 'spiro_inhaler_last_hour_2.0', 'spiro_smoked_last_hour_2.0'
                   'time_outdoors_summer_0.0', 'time_outdoors_winter_0.0', 'time_outdoors_summer_2.0', 'time_outdoors_winter_2.0',
                   'Chest_pain_0.0', 'Chest_pain_walking_normally_0.0', 'Chest_pain_walking_ceases_standing_still_0.0', 'Chest_pain_walking_uphill_or_hurrying_0.0', 
                   'Chest_pain_2.0', 'Chest_pain_walking_normally_2.0', 'Chest_pain_walking_ceases_standing_still_2.0', 'Chest_pain_walking_uphill_or_hurrying_2.0'
                   'whistling_chest_last_year_0.0', 'shortness_breath_walking_0.0', 'whistling_chest_last_year_2.0', 'shortness_breath_walking_2.0',
                   'leisure_activities_0.0', 'leisure_activities_0.1', 'leisure_activities_0.2', 'leisure_activities_0.3', 'leisure_activities_0.4', 
                   'leisure_activities_2.0', 'leisure_activities_2.1', 'leisure_activities_2.2', 'leisure_activities_2.3', 'leisure_activities_2.4' 
                   'Past_tobacco_smoking_2.0', 'Aclohol_intake_vs_10_prev_years_2.0', 'Former_alcohol_drinker_2.0', 'Ever_Smoked_2.0', 'Current_tobacco_smoking_2.0',
                   'Cooked_vegetable_intake_0.0', 'Salad_raw_vegetable_intake_0.0', 'Processed_meat_intake_0.0', 'Cooked_vegetable_intake_2.0', 
                   'Salad_raw_vegetable_intake_2.0', 'Processed_meat_intake_2.0',
                   'Former_alcohol_drinker_0.0', 'Ever_Smoked_0.0', 'Current_tobacco_smoking_0.0', 'Past_tobacco_smoking_0.0', 'Aclohol_intake_vs_10_prev_years_0.0']


In [5]:
numeric_df = process_columns_for_rf(df_metadata.copy(), columns_to_drop, columns_to_code, nan_threshold=1)

# Define the subjects you want to remove
# numeric_df['Subject'] = df_metadata['Subject'].copy()
numeric_df = numeric_df.reset_index(drop=False)

# Optionally, inspect the resulting DataFrame.
print("Processed DataFrame columns:")
print(numeric_df.columns)
print(numeric_df.dtypes)
print(numeric_df.shape)

Processed DataFrame columns:
Index(['Subject', 'Age_last_oral_contraceptive_0.0', 'Sex_0.0',
       'Pacemaker_0.0', 'Ethnicity_0.0', 'BMI_0.0', 'Weight_0.0',
       'Townsend_deprivation_index_0.0', 'Age_last_oral_contraceptive_2.0',
       'Pacemaker_2.0',
       ...
       'dyspnea', 'hp', 'hf', 'diabetes', 'sf_af', 'sf_hp', 'sf_hf',
       'sf_diabetes', 'Age_Death', 'Date_Death'],
      dtype='object', length=369)
Subject                             object
Age_last_oral_contraceptive_0.0    float64
Sex_0.0                            float64
Pacemaker_0.0                      float64
Ethnicity_0.0                      float64
                                    ...   
sf_hp                              float64
sf_hf                              float64
sf_diabetes                        float64
Age_Death                          float64
Date_Death                         float64
Length: 369, dtype: object
(41559, 369)


In [6]:
columns_to_drop = ['Age_last_oral_contraceptive_0.0', 'Pacemaker_0.0', 'Ethnicity_0.0', 'BMI_0.0', 'Weight_0.0',
                   'Smoking_status_0.0', 'Drinker_status_0.0', 'Former_alcohol_drinker_0.0', 'Ever_Smoked_0.0', 'Current_tobacco_smoking_0.0', 
                   'Past_tobacco_smoking_0.0', 'Aclohol_intake_vs_10_prev_years_0.0', 'Overall_health_rating_0.0', 'Long_standing_illness_disability_0.0', 
                   'Falls_last_years_0.0', 'Weight_change_last_year_0.0', 'health_satisfaction_0.0', 'leisure_activities_0.0', 'leisure_activities_0.1', 
                   'leisure_activities_0.2', 'leisure_activities_0.3', 'left_hand_grip_strength_0.0', 'right_hand_grip_strength_0.0','sleep_duration_0.0', 
                   'snoring_0.0', 'age_last_used_oral_contraceptive_pill_0.0', 'age_when_periods_started_0.0', 'had_menopause_0.0', 
                   'number_of_live_births_0.0', 'age_at_first_live_birth_0.0', 'age_at_last_live_birth_0.0', 
                   'ever_had_miscarriage_stillbirths_or_termination_0.0', 'ever_taken_oral_contraceptive_pill_0.0', 'age_started_oral_contraceptive_pill_0.0', 
                   'ever_used_hormone_replacement_therapy_0.0', 'age_at_hysterectomy_0.0', 'bilateral_oophorectomy_0.0', 
                   'age_started_hormone_replacement_therapy_0.0', 'age_last_used_hormone_replacement_therapy_0.0', 'age_at_menopause_0.0', 
                   'ever_had_hysterectomy_0.0', 'time_since_last_menstrual_period_0.0', 'length_of_menstrual_cycle_0.0', 'menstruating_today_0.0', 
                   'number_of_stillbirths_0.0', 'number_of_miscarriages_0.0', 'number_of_terminations_0.0', 'age_of_primiparous_women_at_birth_of_child_0.0', 
                   'age_at_bilateral_oophorectomy_0.0', 'ecg_ex_heart_rate_0.0', 'ecg_ex_heart_rate_0.1', 'ecg_ex_heart_rate_0.2', 'ecg_ex_heart_rate_0.3', 
                   'ecg_ex_heart_rate_0.4', 'ecg_ex_heart_rate_0.5', 'ecg_ex_heart_rate_0.6', 'ecg_ex_heart_rate_0.7', 'ecg_ex_heart_rate_0.8', 
                   'ecg_ex_heart_rate_0.9', 'ecg_ex_heart_rate_0.10', 'ecg_ex_heart_rate_0.11', 'ecg_ex_heart_rate_0.12', 'ecg_ex_heart_rate_0.13', 
                   'ecg_ex_heart_rate_0.14', 'ecg_ex_heart_rate_0.15', 'ecg_ex_heart_rate_0.16', 'ecg_ex_heart_rate_0.17', 'ecg_ex_heart_rate_0.18', 
                   'ecg_ex_heart_rate_0.19', 'ecg_ex_heart_rate_0.20', 'ecg_ex_heart_rate_0.21', 'ecg_ex_heart_rate_0.22', 'ecg_ex_heart_rate_0.23', 
                   'ecg_ex_heart_rate_0.24', 'ecg_ex_heart_rate_0.25', 'ecg_ex_heart_rate_0.26', 'ecg_ex_heart_rate_0.27', 'ecg_ex_heart_rate_0.28', 
                   'ecg_ex_heart_rate_0.29', 'ecg_ex_heart_rate_0.30', 'ecg_ex_heart_rate_0.31', 'ecg_ex_heart_rate_0.32', 'ecg_ex_heart_rate_0.33', 
                   'ecg_ex_heart_rate_0.34', 'ecg_ex_heart_rate_0.35', 'ecg_ex_heart_rate_0.36', 'ecg_ex_heart_rate_0.37', 'ecg_ex_heart_rate_0.38', 
                   'ecg_ex_heart_rate_0.39', 'ecg_ex_heart_rate_0.40', 'ecg_ex_heart_rate_0.41', 'ecg_ex_heart_rate_0.42', 'ecg_ex_heart_rate_0.43', 
                   'ecg_ex_heart_rate_0.44', 'ecg_ex_heart_rate_0.45', 'ecg_ex_heart_rate_0.46', 'ecg_ex_heart_rate_0.47', 'ecg_ex_heart_rate_0.48', 
                   'ecg_ex_heart_rate_0.49', 'ecg_ex_heart_rate_0.50', 'ecg_ex_heart_rate_0.51', 'ecg_ex_heart_rate_0.52', 'ecg_ex_heart_rate_0.53', 
                   'ecg_ex_heart_rate_0.54', 'ecg_ex_heart_rate_0.55', 'ecg_ex_heart_rate_0.56', 'ecg_ex_heart_rate_0.57', 'ecg_ex_heart_rate_0.58', 
                   'ecg_ex_heart_rate_0.59', 'ecg_ex_heart_rate_0.60', 'ecg_ex_heart_rate_0.61', 'ecg_ex_heart_rate_0.62', 'ecg_ex_heart_rate_0.63', 
                   'ecg_ex_heart_rate_0.64', 'ecg_ex_heart_rate_0.65', 'ecg_ex_heart_rate_0.66', 'ecg_ex_heart_rate_0.67', 'ecg_ex_heart_rate_0.68', 
                   'ecg_ex_heart_rate_0.69', 'ecg_ex_heart_rate_0.70', 'ecg_ex_heart_rate_0.71', 'ecg_ex_heart_rate_0.72', 'ecg_ex_heart_rate_0.73', 
                   'ecg_ex_heart_rate_0.74', 'ecg_ex_heart_rate_0.75', 'ecg_ex_heart_rate_0.76', 'ecg_ex_heart_rate_0.77', 'ecg_ex_heart_rate_0.78', 
                   'ecg_ex_heart_rate_0.79', 'ecg_ex_heart_rate_0.80', 'ecg_ex_heart_rate_0.81', 'ecg_ex_heart_rate_0.82', 'ecg_ex_heart_rate_0.83', 
                   'ecg_ex_heart_rate_0.84', 'ecg_ex_heart_rate_0.85', 'ecg_ex_heart_rate_0.86', 'ecg_ex_heart_rate_0.87', 'ecg_ex_heart_rate_0.88', 
                   'ecg_ex_heart_rate_0.89', 'ecg_ex_heart_rate_0.90', 'ecg_ex_heart_rate_0.91', 'ecg_ex_heart_rate_0.92', 'ecg_ex_heart_rate_0.93', 
                   'ecg_ex_heart_rate_0.94', 'ecg_ex_heart_rate_0.95', 'ecg_ex_heart_rate_0.96', 'ecg_ex_heart_rate_0.97', 'ecg_ex_heart_rate_0.98', 
                   'ecg_ex_heart_rate_0.99', 'ecg_ex_heart_rate_0.100', 'ecg_ex_heart_rate_0.101', 'ecg_ex_heart_rate_0.102', 'ecg_ex_heart_rate_0.103', 
                   'ecg_ex_heart_rate_0.104', 'ecg_ex_heart_rate_0.105', 'ecg_ex_heart_rate_0.106','ecg_ex_heart_rate_0.107', 'ecg_ex_heart_rate_0.108', 
                   'ecg_ex_max_workload_0.0', 'ecg_ex_max_heart_rate_0.0', 'ecg_ex_target_rate_achieved_0.0', 'art_stiff_pulse_rate_0.0', 
                   'art_stiff_reflection_index_0.0', 'art_stiff_peak_to_peak_time_0.0', 'art_stiff_position_peak_0.0', 'art_stiff_position_notch_0.0', 
                   'art_stiff_position_shoulder_0.0', 'art_stiff_absence_notch_0.0', 'art_stiff_stiffness_index_0.0', 'whistling_chest_last_year_0.0', 
                   'shortness_breath_walking_0.0', 'Chest_pain_0.0', 'Chest_pain_walking_normally_0.0', 'Chest_pain_walking_ceases_standing_still_0.0', 
                   'Chest_pain_walking_uphill_or_hurrying_0.0', 'Cooked_vegetable_intake_0.0', 'Salad_raw_vegetable_intake_0.0', 'Processed_meat_intake_0.0', 
                   'Cooked_vegetable_intake_2.0', 'Salad_raw_vegetable_intake_2.0', 'Processed_meat_intake_2.0', 'time_outdoors_summer_0.0', 
                   'time_outdoors_winter_0.0', 'spiro_caffeine_last_hour_0.0', 'spiro_inhaler_last_hour_0.0', 'spiro_smoked_last_hour_0.0', 
                   'spiro_fev1_predicted_0.0', 'spiro_fev1_percentage_0.0', 'spiro_fev1_zscore_0.0', 'spiro_fvc_zscore_0.0', 
                   'spiro_fev1_fvc_ratio_zscores_0.0', 'bsa_img_2.0', 'af', 'dyspnea', 'hp', 'hf', 'diabetes', 'sf_af', 'sf_hp', 'sf_hf', 'sf_diabetes', 
                   'Age_Death', 'Date_Death', 'Alcohol_int_freq_0.0', 'PulseRate_0.0', 'DBP_0.0',
                   'Alcohol_int_freq_2.0', 'Smoking_status_2.0', 'Drinker_status_2.0', 'Former_alcohol_drinker_2.0', 'Ever_Smoked_2.0', 'Current_tobacco_smoking_2.0', 
                   'Past_tobacco_smoking_2.0', 'Aclohol_intake_vs_10_prev_years_2.0',
                   'ecg_rest_automated_diag_2.0', 'ecg_rest_automated_diag_2.1', 'ecg_rest_automated_diag_2.2', 'ecg_rest_automated_diag_2.3', 
                   'ecg_rest_automated_diag_2.4', 'ecg_rest_automated_diag_2.5', 'ecg_rest_automated_diag_2.6', 'ecg_rest_automated_diag_2.7', 
                   'ecg_rest_automated_diag_2.8', 'ecg_rest_automated_diag_2.9', 'ecg_rest_automated_diag_2.10', 'ecg_rest_automated_diag_2.11', 
                   'ecg_rest_automated_diag_2.12', 'ecg_rest_automated_diag_2.13', 'ecg_rest_automated_diag_2.14', 'ecg_rest_flag_2.0', 'pwa_number_beats_2.0',
                   'pwa_vicorder_results_plausible_2.0', 'ipaq_activity_0.0', 'leisure_activities_0.4','SBP_0.0',
                   'leisure_activities_2.0', 'leisure_activities_2.1', 'leisure_activities_2.2', 'leisure_activities_2.3', 'leisure_activities_2.4',
                   'lv_ef_2.0', 'lv_edv_2.0', 'lv_esv_2.0', 'lv_sv_2.0', 'cardiac_output_2.0', 'cardiac_index_2.0', 'avg_heart_rate_2.0',
                   'Weight_Img_2.0', 'Overall_health_rating_2.0', 'Long_standing_illness_disability_2.0', 
                   'Falls_last_years_2.0', 'Weight_change_last_year_2.0', 'neuroticism_score_0.0', 'health_satisfaction_2.0',
                   'sleep_duration_2.0', 'snoring_2.0', 'spiro_caffeine_last_hour_2.0', 'spiro_inhaler_last_hour_2.0',
                   'time_outdoors_summer_2.0', 'time_outdoors_winter_2.0']

numeric_df.drop(columns=columns_to_drop, inplace=True)
print(list(numeric_df.columns))
print(numeric_df.shape)

['Subject', 'Sex_0.0', 'Townsend_deprivation_index_0.0', 'Age_last_oral_contraceptive_2.0', 'Pacemaker_2.0', 'Height', 'Ethnicity_2.0', 'Age_2.0', 'PulseRate_2.0', 'DBP_2.0', 'SBP_2.0', 'left_hand_grip_strength_2.0', 'right_hand_grip_strength_2.0', 'age_last_used_oral_contraceptive_pill_2.0', 'age_when_periods_started_2.0', 'had_menopause_2.0', 'number_of_live_births_2.0', 'age_at_first_live_birth_2.0', 'age_at_last_live_birth_2.0', 'ever_had_miscarriage_stillbirths_or_termination_2.0', 'ever_taken_oral_contraceptive_pill_2.0', 'age_started_oral_contraceptive_pill_2.0', 'ever_used_hormone_replacement_therapy_2.0', 'age_at_hysterectomy_2.0', 'bilateral_oophorectomy_2.0', 'age_started_hormone_replacement_therapy_2.0', 'age_last_used_hormone_replacement_therapy_2.0', 'age_at_menopause_2.0', 'ever_had_hysterectomy_2.0', 'time_since_last_menstrual_period_2.0', 'length_of_menstrual_cycle_2.0', 'menstruating_today_2.0', 'number_of_stillbirths_2.0', 'number_of_miscarriages_2.0', 'number_of_ter

In [7]:
# Load the biomarkers file and merge with the numeric DataFrame
biomarkers_filename = os.path.join(derivatives_path, 'biomarkers.csv')
df_biomarkers = pd.read_csv(biomarkers_filename, index_col=0)

df_biomarkers.drop(columns=['Height', 'Weight', 'BMI'], inplace=True)

df_merged = pd.merge(df_biomarkers, numeric_df, on='Subject', how='left')
df_merged.to_csv(os.path.join(derivatives_path, 'metadata_participants_biomarkers.csv'))

numeric_df_filtered = numeric_df[numeric_df['Subject'].isin(df_biomarkers['Subject'])]
numeric_df_filtered.to_csv(os.path.join(derivatives_path, 'metadata_participants_numeric.csv'), index=False)
